# Imports

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

# Load in Data

In [2]:
pickle_to_load = "../../FantasyData/data-frames/df_wr_college_2000_2023.pkl"

In [3]:
df_pos = pd.read_pickle(pickle_to_load)

### For saving Data Later

In [4]:
save_path = "../../interactive-2.0/WR/draft-capital/"

# Clean Up Data

In [33]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

In [34]:
# Mapping for acronyms and name changes
team_mapping = {
    'NYG': 'New York Giants',
    'DEN': 'Denver Broncos',
    'NWE': 'New England Patriots',
    'OAK': 'Las Vegas Raiders',  # Mapped directly to Las Vegas Raiders
    'MIN': 'Minnesota Vikings',
    'GNB': 'Green Bay Packers',
    'CIN': 'Cincinnati Bengals',
    'KAN': 'Kansas City Chiefs',
    'LAR': 'Los Angeles Rams',
    'MIA': 'Miami Dolphins',
    'ARI': 'Arizona Cardinals',
    'SEA': 'Seattle Seahawks',
    'NOR': 'New Orleans Saints',
    'CLE': 'Cleveland Browns',
    'ATL': 'Atlanta Falcons',
    'TAM': 'Tampa Bay Buccaneers',
    'DET': 'Detroit Lions',
    'IND': 'Indianapolis Colts',
    'PHI': 'Philadelphia Eagles',
    'CAR': 'Carolina Panthers',
    'LAC': 'Los Angeles Chargers',
    'JAX': 'Jacksonville Jaguars',
    'TEN': 'Tennessee Titans',
    'St. Louis Rams': 'Los Angeles Rams',
    'San Diego Chargers': 'Los Angeles Chargers',
    'Oakland Raiders': 'Las Vegas Raiders',
    'Washington Redskins': 'Washington Commanders',
    'Washington Football Team': 'Washington Commanders'
}

# Replace the values in the 'draft_team' column
df_clean['draft_team'] = df_clean['draft_team'].replace(team_mapping)

# Get Draft Capital Success Rate By Round

Grab the best (lower tier is better) tier a player has been in, for each round of the draft:

In [35]:
tier_counts = df_clean.groupby(['draft_team', 'draft_round', 'player_name'])['tier'].min().reset_index()

In [48]:
tier_counts.head(10)

,draft_team,draft_round,player_name,tier
0,Arizona Cardinals,1.0,Larry Fitzgerald,1
1,Arizona Cardinals,1.0,Michael Floyd,3
2,Arizona Cardinals,2.0,Andy Isabella,10
3,Arizona Cardinals,2.0,Anquan Boldin,1
4,Arizona Cardinals,2.0,Christian Kirk,1
5,Arizona Cardinals,2.0,Rondale Moore,6
6,Arizona Cardinals,3.0,Andre Roberts,3
7,Arizona Cardinals,3.0,Chad Williams,11
8,Arizona Cardinals,3.0,Damaris Johnson,8
9,Arizona Cardinals,3.0,Early Doucet,4


Then group this data frame by the draft round to get the total number of players in each round

In [37]:
total_players_per_round = tier_counts \
    .groupby(['draft_team', 'draft_round'])['player_name'] \
    .count() \
    .reset_index(name='total_players')

Grab the number of each players in each tier, by their draft round:

In [38]:
players_in_tier = tier_counts.groupby(['draft_team', 'draft_round', 'tier']).size().reset_index(name='players_in_tier')

Merge the dataframes together:

In [39]:
merged_data = pd.merge(players_in_tier, total_players_per_round, on=['draft_team', 'draft_round'])
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

Tier percentagers by draft round:

In [40]:
merged_data.head(5)

,draft_team,draft_round,tier,players_in_tier,total_players,percentage
0,Arizona Cardinals,1.0,1,1,2,50.0
1,Arizona Cardinals,1.0,3,1,2,50.0
2,Arizona Cardinals,2.0,1,2,4,50.0
3,Arizona Cardinals,2.0,6,1,4,25.0
4,Arizona Cardinals,2.0,10,1,4,25.0


# Analysis

### Success by Tier

In [54]:
df_success_by_tier = merged_data.query("tier <= 4").groupby(["draft_team", "draft_round"]).apply(lambda x: x)

/tmp/ipykernel_22963/299084279.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_success_by_tier = merged_data.query("tier <= 4").groupby(["draft_team", "draft_round"]).apply(lambda x: x)


In [55]:
df_success_by_tier.query("percentage > 50 and total_players >= 3")

,,,draft_team,draft_round,tier,players_in_tier,total_players,percentage
draft_team,draft_round,,,,,,,
Atlanta Falcons,1.0,14,Atlanta Falcons,1.0,1,3,5,60.000000
Cincinnati Bengals,1.0,68,Cincinnati Bengals,1.0,1,2,3,66.666667
Green Bay Packers,2.0,135,Green Bay Packers,2.0,1,4,6,66.666667
Houston Texans,1.0,150,Houston Texans,1.0,1,2,3,66.666667
Miami Dolphins,1.0,234,Miami Dolphins,1.0,1,2,3,66.666667
Pittsburgh Steelers,3.0,330,Pittsburgh Steelers,3.0,1,3,5,60.000000


In [56]:
df_success_by_tier.query("draft_team == 'Pittsburgh Steelers'")

draft_team  draft_round  tier  \
draft_team          draft_round                                               
Pittsburgh Steelers 1.0         324  Pittsburgh Steelers          1.0     1   
                                325  Pittsburgh Steelers          1.0     2   
                    2.0         326  Pittsburgh Steelers          2.0     1   
                                327  Pittsburgh Steelers          2.0     2   
                                328  Pittsburgh Steelers          2.0     3   
                    3.0         330  Pittsburgh Steelers          3.0     1   
                                331  Pittsburgh Steelers          3.0     4   
                    4.0         333  Pittsburgh Steelers          4.0     4   
                    6.0         335  Pittsburgh Steelers          6.0     1   

                                     players_in_tier  total_players  \
draft_team          draft_round                                       
Pittsburgh Steelers 1.0         324                1              2   
                                325                1              2   
                    2.0         326                1              4   
                                327                1              4   
                                328                1              4   
                    3.0         330                3              5   
                                331                1              5   
                    4.0         333                1              2   
                    6.0         335                1              1   

                                     percentage  
draft_team          draft_round                  
Pittsburgh Steelers 1.0         324        50.0  
                                325        50.0  
                    2.0         326        25.0  
                                327        25.0  
                                328        25.0  
                    3.0         330        60.0  
                                331        20.0  
                    4.0         333        50.0  
                    6.0         335       100.0

### General Success

In [64]:
df_success = merged_data.query("tier <= 4 and total_players >= 3") \
    .groupby(["draft_team", "draft_round"]).agg({"percentage": "sum"})

In [65]:
df_success.query("percentage > 50")

percentage
draft_team           draft_round            
Atlanta Falcons      1.0          100.000000
Chicago Bears        2.0           66.666667
Cincinnati Bengals   1.0           66.666667
                     2.0          100.000000
Dallas Cowboys       3.0           66.666667
Green Bay Packers    2.0           83.333333
Houston Texans       1.0          100.000000
Indianapolis Colts   2.0           66.666667
                     3.0          100.000000
Jacksonville Jaguars 2.0          100.000000
                     4.0           75.000000
Las Vegas Raiders    1.0           66.666667
                     5.0           66.666667
Los Angeles Chargers 3.0           66.666667
Miami Dolphins       1.0          100.000000
Minnesota Vikings    1.0           80.000000
New Orleans Saints   1.0          100.000000
New York Giants      2.0           75.000000
Philadelphia Eagles  1.0           75.000000
                     2.0           66.666667
Pittsburgh Steelers  2.0           75.000000
                     3.0           80.000000
San Francisco 49ers  1.0           66.666667
Seattle Seahawks     2.0           80.000000
Tennessee Titans     1.0           75.000000